In [ ]:
!pip install JPype1==1.3.0 #pip install JPype1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 820.3/820.3 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for JPype1: filename=JPype1-1.3.0-cp310-cp310-linux_x86_64.whl size=454408 sha256=bcfe8aaa9fba45c9e2115db61a3826a5057146821aed06b7836e0dc19cbbf545
  Stored in directory: /root/.cache/pip/wheels/f5/c7/8f/c97c6c9868c256c8d17dabb772a2ca9002dcff2912fa8d7d58
Successfully built JPype1


In [ ]:
# Zemberek'i kullanabillmek için ilk önce java kütüphenelerini kullanmamızı sağlayan kütüphaneyi ekleyelim
import jpype
from jpype import JClass

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import zipfile

def read_jar(jar_file):
    zf = zipfile.ZipFile(jar_file, 'r')
    try:
        lst = zf.infolist()
        for zi in lst:
            fn = zi.filename
            # print(fn)
    finally:
        zf.close()

read_jar("/content/drive/MyDrive/zemberek-full.jar")

In [ ]:
# Zemberek'i çalıştıralım
jar = "/content/drive/MyDrive/zemberek-full.jar"  # Zemberek'in yolu
if not jpype.isJVMStarted():
    jpype.startJVM(jvmpath=jpype.getDefaultJVMPath(), classpath=jar)
    print("tamamdır")

tamamdır


In [ ]:
# Zemberekten gerekli kütüphaneleri import edelim
try:
    TurkishTokenizer = JClass('zemberek.tokenization.TurkishTokenizer')
    Token = JClass('zemberek.tokenization.Token')
    TurkishMorphology = JClass('zemberek.morphology.TurkishMorphology')
    TurkishSpellChecker = JClass('zemberek.normalization.TurkishSpellChecker')
    WordAnalysis =  JClass('zemberek.morphology.analysis.WordAnalysis')
    TurkishSentenceExtractor = JClass('zemberek.tokenization.TurkishSentenceExtractor')
except:
    print("yanlış yol")

In [ ]:
morphology = TurkishMorphology.createWithDefaults()
tokenizer = TurkishTokenizer.ALL
spell_checker = TurkishSpellChecker(morphology)
extractor = TurkishSentenceExtractor.DEFAULT

In [ ]:
stopwords = [x.strip() for x in open('/content/drive/MyDrive/stop-words.tr.txt','r', encoding="UTF8").read().split('\n')]

In [ ]:
#Verilerimizi almak ve işlemek için gerekli kütüphaneler
import pandas as pd
import numpy as np
import string

In [ ]:
import pandas as pd
import os

# Dosyaların bulunduğu dizinler
kose_yazilari_dizinler = [
    "/content/drive/MyDrive/KOSE_YAZILARİ/AHMET_HAKAN/",
    "/content/drive/MyDrive/KOSE_YAZILARİ/ASU_MARO/",
    "/content/drive/MyDrive/KOSE_YAZILARİ/ERMAN_TOROGLU/",
    "/content/drive/MyDrive/KOSE_YAZILARİ/FATİH_ALTAYLİ/",
    "/content/drive/MyDrive/KOSE_YAZILARİ/SAVAS_OZBEY/"
]

# Tüm DataFrame'leri birleştirmek için bir liste oluştur
tum_df_listesi = []

for kose_yazilari_dizin in kose_yazilari_dizinler:
    # Dosya adları
    dosya_adlari = [f"{kose_yazilari_dizin}{i}.txt" for i in range(1, 51)]

    # Her bir dosyadaki metinleri oku ve bir liste oluştur
    metin_listesi = []
    for dosya_adı in dosya_adlari:
        with open(dosya_adı, "r", encoding="utf-8") as dosya:
            metin = dosya.read()
            metin_listesi.append(metin)

    # DataFrame oluştur
    df = pd.DataFrame({"title": [kose_yazilari_dizin.split("/")[-2]] * 50, "text": metin_listesi})

    # DataFrame'i liste içine ekle
    tum_df_listesi.append(df)

# Tüm DataFrame'leri birleştir
b_df = pd.concat(tum_df_listesi, axis=0)

# İlk beş satırı görüntüle
b_df.head(250)


,title,text
0,AHMET_HAKAN,Filistin’i bahane edip Cumhuriyet’i kutlamak i...
1,AHMET_HAKAN,“SİHA’mızı vurdular” gerçek bir özet değil\n\n...
2,AHMET_HAKAN,Polat’lar Talu’lar\n\n\nBuradan ilgili mercile...
3,AHMET_HAKAN,Ölü bebeklerle yapılan bir basın toplantısının...
4,AHMET_HAKAN,Yepyeni bir uyarı yöntemi: Tek kare fotoğraf i...
...,...,...
45,SAVAS_OZBEY,"Oh ne rahat, tek tabanca seyahat\n\nBirçok ins..."
46,SAVAS_OZBEY,Ben güzele güzel demem güzel kendi olmayınca\n...
47,SAVAS_OZBEY,Kendi küçük keyfi büyük... Sahillerden en beğe...
48,SAVAS_OZBEY,"Serdar Ortaç’a açık mektup\n\nAbi yerleş, nere..."


In [ ]:
def dataCleaning(text):
    #Satır boşluklarını kaldırır, bütün metni küçük harfe çevirir, noktalama işaretlerini kaldırır
    text = text.replace("\n", " ")
    text = text.lower()
    text = "".join([i for i in text if (i.isalnum() or i == " ")])
    return " ".join(text.split())


def removeStopwords(text):
    # stopwords dosyamızda bulunan kelimeleri metinden kaldırır
    for word in text:
        if word in stopwords or word in string.whitespace:
            text.remove(word)
    return text

In [ ]:
def wordTokenize(text):
    #Önişlenmiş metini kelimelere ayırır ve stopword'leri kaldırır
    text = text.split(" ")
    text = removeStopwords(text)
    return text

def sentTokenize(text):
    #Önişlenmemiş metini cümlelerine ayırır, bunişlem sırasında önişleme yapar ve stopword'leri kaldırır
    sent_list = []
    text = text.replace("\"", "")

    for sentence in text.split("."):
        sentence = dataCleaning(sentence)
        sentence = removeStopwords(sentence.split(" "))
        if len(sentence) == 0:
            continue
        else:
            sent_list.append(" ".join(sentence))
    return sent_list

In [ ]:


# unk kelime sayısı da nitelik olarak eklenmeli -yabancı kelime sayısı özniteliği-
def lemmas(word_list):
    #Kelime token'larından kök(kelimenin en saf hali) tokenları oluştururlur
    lemma = []
    for word in word_list:
        result = str(morphology.analyzeAndDisambiguate(word).bestAnalysis()[0].getLemmas()[0])
        if result == "UNK":
            lemma.append(result)
        else:
            lemma.append(result)
    return lemma

def wtDist(wt):
    #Kelimelerin dağılımları
    wt_dist = dict()
    wt_dist.fromkeys(set(wt))
    for i in set(wt):
        wt_dist[i] = wt.count(i)
    return wt_dist

def wtLenDist(wt):
    #Kelimelerin harf olarak uzunluk dağılımlarını çıkarır
    wt_len = [len(str(word)) for word in wt]
    wt_len_dist = dict()
    wt_len_dist.fromkeys(range(1, 29))
    for i in range(0, 29):
        wt_len_dist[i] = wt_len.count(i)
    return wt_len_dist

def stLenDist(st):
    #Cümlelerin kelime olarak uzunluk dağılımlarını çıkarır
    st_len = [len(wordTokenize(sent)) for sent in st]
    st_len_dist = dict()
    st_len_dist.fromkeys(range(1, 29))
    for i in range(0, 29):
        st_len_dist[i] = st_len.count(i)
    return st_len_dist

def typeTokenRatio(wt):
    #Kelimenin toplam kelime sayısına oranı
    return len(set(wt))/len((wt))

def avgWtLen(wt):
    #Ortalama kelime uzunluğu
    return sum(len(word) for word in wt)/len(wt)


def avgStLen(st):
    #Kelime olarak ortalama cümle uzunluğu
    return sum(len(wordTokenize(sent)) for sent in st)/len(st)


def puncNum(txt):
    #Noktalama işareti sayısı
    return len([x for x in txt if x in string.punctuation])


def numStopwords(clean_text):
    #Stopword kullanma sayısı
    return len([w for w in clean_text.split() if w in stopwords])


def numUpper(raw_text):
    #Tamamı büyük harf olan kelime sayısı
    p = string.punctuation + "’" + "“" + "”"
    raw_text = ''.join(' ' if c in p else c for c in raw_text)
    return len([w for w in raw_text.split() if str(w).isupper()])

In [ ]:
b_df[b_df['title'].str.contains('AHMET_HAKAN')].shape

(50, 2)

In [ ]:
b_df['clean_text'] = b_df['text'].apply(lambda x : dataCleaning(x))
b_df['word_token'] = b_df['clean_text'].apply(lambda x : wordTokenize(x))
b_df['sent_token'] = b_df['text'].apply(lambda x : sentTokenize(x))
b_df['lemma_token'] = b_df['word_token'].apply(lambda x : lemmas(x))
b_df['ltDist'] = b_df['lemma_token'].apply(lambda x : wtDist(x))
b_df['wtLenDist'] = b_df['word_token'].apply(lambda x : wtLenDist(x))
b_df['stLenDist'] = b_df['sent_token'].apply(lambda x : stLenDist(x))
b_df['ttr'] = b_df['word_token'].apply(lambda x : typeTokenRatio(x))
b_df['lttr'] = b_df['lemma_token'].apply(lambda x : typeTokenRatio(x))
b_df['avgWtLen'] = b_df['word_token'].apply(lambda x : avgWtLen(x))
b_df['avgStLen'] = b_df['sent_token'].apply(lambda x : avgStLen(x))
b_df['puncNum'] = b_df['text'].apply(lambda x : puncNum(x))
b_df['numStopwords'] = b_df['clean_text'].apply(lambda x : numStopwords(x))
b_df['numUpper'] = b_df['text'].apply(lambda x : numUpper(x))

# yazarları tam sayı olarak kodlar ve tam sayı-yazar sözlüğü oluşturur
b_df['target'], mapping = pd.factorize(b_df.title)

In [ ]:
mapping[1:5]

Index(['ASU_MARO', 'ERMAN_TOROGLU', 'FATİH_ALTAYLİ', 'SAVAS_OZBEY'], dtype='object')

In [ ]:
b_df.head()

,title,text,clean_text,word_token,sent_token,lemma_token,ltDist,wtLenDist,stLenDist,ttr,lttr,avgWtLen,avgStLen,puncNum,numStopwords,numUpper,target
0,AHMET_HAKAN,Filistin’i bahane edip Cumhuriyet’i kutlamak i...,filistini bahane edip cumhuriyeti kutlamak ist...,"[filistini, bahane, edip, cumhuriyeti, kutlama...",[filistini bahane edip cumhuriyeti kutlamak is...,"[filistin, bahane, et, cumhuriyet, kutla, iste...","{'kardeş': 1, 'itidal': 2, 'süreç': 2, 'bin': ...","{0: 0, 1: 5, 2: 14, 3: 22, 4: 29, 5: 49, 6: 66...","{0: 0, 1: 0, 2: 3, 3: 2, 4: 3, 5: 4, 6: 8, 7: ...",0.817778,0.577778,7.262222,9.488889,123,140,27,0
1,AHMET_HAKAN,“SİHA’mızı vurdular” gerçek bir özet değil\n\n...,sihamızı vurdular gerçek bir özet değil eğer i...,"[sihamızı, vurdular, gerçek, özet, eğer, de, ö...",[sihamızı vurdular gerçek özet eğer de özet ya...,"[UNK, vur, gerçek, özet, eğer, de, özet, yap, ...","{'halk': 1, 'video': 2, 'hal': 2, 'yorum': 1, ...","{0: 0, 1: 0, 2: 7, 3: 19, 4: 37, 5: 55, 6: 67,...","{0: 0, 1: 2, 2: 10, 3: 7, 4: 5, 5: 8, 6: 8, 7:...",0.743649,0.531178,7.203233,6.318182,166,124,60,0
2,AHMET_HAKAN,Polat’lar Talu’lar\n\n\nBuradan ilgili mercile...,polatlar talular buradan ilgili mercilere sesl...,"[polatlar, talular, ilgili, mercilere, sesleni...",[polatlar talular ilgili mercilere sesleniyoru...,"[polat, talu, ilgili, merci, seslen, dilan, po...","{'bol': 1, 'zombi': 1, 'halk': 1, 'medeniyet':...","{0: 0, 1: 0, 2: 1, 3: 30, 4: 21, 5: 56, 6: 58,...","{0: 1, 1: 1, 2: 3, 3: 8, 4: 13, 5: 4, 6: 5, 7:...",0.806916,0.674352,7.089337,5.767857,133,119,34,0
3,AHMET_HAKAN,Ölü bebeklerle yapılan bir basın toplantısının...,ölü bebeklerle yapılan bir basın toplantısının...,"[ölü, bebeklerle, yapılan, basın, toplantısını...",[ölü bebeklerle yapılan basın toplantısının fo...,"[ölü, bebek, yap, basın, toplantı, fotoğraf, h...","{'hal': 1, 'ol': 3, 'ardından': 1, 'şöyle': 1,...","{0: 0, 1: 0, 2: 5, 3: 33, 4: 17, 5: 61, 6: 47,...","{0: 0, 1: 4, 2: 2, 3: 7, 4: 7, 5: 6, 6: 6, 7: ...",0.761773,0.573407,7.202216,6.415094,131,123,26,0
4,AHMET_HAKAN,Yepyeni bir uyarı yöntemi: Tek kare fotoğraf i...,yepyeni bir uyarı yöntemi tek kare fotoğraf ik...,"[uyarı, yöntemi, kare, fotoğraf, net, açıklama...",[uyarı yöntemi kare fotoğraf kısa net açıklama...,"[uyarı, yöntem, kare, fotoğraf, net, açıkla, t...","{'açı': 1, 'iran': 1, 've': 2, 'uyarı': 11, 'a...","{0: 0, 1: 5, 2: 7, 3: 51, 4: 56, 5: 113, 6: 78...","{0: 0, 1: 4, 2: 6, 3: 8, 4: 12, 5: 4, 6: 10, 7...",0.666081,0.465729,6.608084,7.287671,153,203,77,0


In [ ]:
#Modelimiz yazarları tahin edecek, kodlanmış kısımı y'ye atıyoruz.
y = np.array(b_df.target)
#Öznitelikler x'e atılır.
x = b_df.loc[:, 'text' : 'numUpper']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33,random_state = 42)
x_train.reset_index(inplace=True)
x_test.reset_index(inplace=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#train verilerimizi vektörleştirelim
scaler = MinMaxScaler()
normalizedLabels = scaler.fit_transform(x_train.loc[:, 'ttr':'numUpper'])

#kökleri tek bir string metin (corpus) olarak birleştiririz
train_corpus = np.array(x_train.lemma_token)
for i in range(0, len(x_train.lemma_token)):
    train_corpus[i] = " ".join(x_train.lemma_token[i])

len(train_corpus)

#bag of word vektörü
count_vectorize = CountVectorizer()
sparce_matrix = count_vectorize.fit_transform(train_corpus).toarray()
sparce_matrix = (sparce_matrix - sparce_matrix.min())/(sparce_matrix.max() - sparce_matrix.min())

# tf-idf vektörü
tfidf_vector = TfidfVectorizer()
tfidf_matrix = tfidf_vector.fit_transform(train_corpus).toarray()

#kelime-cümle dağılım vektörleri
wtDict_vector = DictVectorizer()
stDict_vector = DictVectorizer()
wtLenDist_matrix = wtDict_vector.fit_transform(x_train.wtLenDist).toarray()
stLenDist_matrix = stDict_vector.fit_transform(x_train.stLenDist).toarray()
wtLenDist_matrix = (wtLenDist_matrix - wtLenDist_matrix.min())/(wtLenDist_matrix.max() - wtLenDist_matrix.min())
stLenDist_matrix = (stLenDist_matrix - stLenDist_matrix.min())/(stLenDist_matrix.max() - stLenDist_matrix.min())

#bütün vektörlerin birleştirelim
attribution = (tfidf_matrix, sparce_matrix, wtLenDist_matrix, stLenDist_matrix, normalizedLabels)
attribution = np.concatenate(attribution, axis = 1)
X_train = attribution

In [ ]:
import pickle
from sklearn.linear_model import LogisticRegression

In [ ]:
# Eğitilen modelin ve özellik çıkarım nesnelerinin yüklenmesi
with open('/content/drive/MyDrive/modell/mapping.pkl', 'rb') as f:
    mapping = pickle.load(f)

with open('/content/drive/MyDrive/modell/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('/content/drive/MyDrive/modell/count_vectorize.pkl', 'rb') as f:
    count_vectorize = pickle.load(f)

with open('/content/drive/MyDrive/modell/tfidf_vector.pkl', 'rb') as f:
    tfidf_vector = pickle.load(f)

with open('/content/drive/MyDrive/modell/wtDict_vector.pkl', 'rb') as f:
    wtDict_vector = pickle.load(f)

with open('/content/drive/MyDrive/modell/stDict_vector.pkl', 'rb') as f:
    stDict_vector = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/modell/stDict_vector.pkl', 'wb') as f:
    pickle.dump(stDict_vector, f)


In [ ]:
#oluştruduğumuz vektör modelleri ile test veri setini vektörleştirelim
test_normalizedLabels = scaler.transform(x_test.loc[:, 'ttr':'numUpper'])

test_corpus = np.array(x_test.lemma_token)
for i in range(0, len(x_test.lemma_token)):
    test_corpus[i] = " ".join(x_test.lemma_token[i])

test_sparce_matrix = count_vectorize.transform(test_corpus).toarray()
test_sparce_matrix = (test_sparce_matrix - test_sparce_matrix.min())/(test_sparce_matrix.max() - test_sparce_matrix.min())

test_tfidf_matrix = tfidf_vector.transform(test_corpus).toarray()

test_wtLenDist_matrix = wtDict_vector.transform(x_test.wtLenDist).toarray()
test_stLenDist_matrix = stDict_vector.transform(x_test.stLenDist).toarray()
test_wtLenDist_matrix = (test_wtLenDist_matrix - test_wtLenDist_matrix.min())/(test_wtLenDist_matrix.max() - test_wtLenDist_matrix.min())
test_stLenDist_matrix = (test_stLenDist_matrix - test_stLenDist_matrix.min())/(test_stLenDist_matrix.max() - test_stLenDist_matrix.min())

test_attribution = (test_tfidf_matrix, test_sparce_matrix, test_wtLenDist_matrix, test_stLenDist_matrix, test_normalizedLabels)
test_attribution = np.concatenate(test_attribution, axis = 1)
X_test = test_attribution

model = LogisticRegression()  # Daha önce eğitilmiş modelinizi kullanalım
# Modeli eğit
model.fit(X_train, y_train)

# Şimdi tahminler yapabiliriz
tahminler = model.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score

# Gerçek verilerle model tarafından yapılan tahminlerin karşılaştırılması
dogruluk = accuracy_score(y_test, tahminler)

print(f'Model Doğruluğu: {dogruluk}')

Model Doğruluğu: 0.927710843373494
